In [1]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader,RandomSampler,SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import random
import json
# import optuna
from torch.nn import functional
import datetime
import gc
import os
import glob
from tqdm import tqdm

In [2]:
all_data = np.load('D:/myfiles/project/bike_prediction/feature_data/tcn_data_3d.npy')
all_data.shape

(753, 3312, 8)

In [4]:
# 【站点数量，序列长度，特征数量】
class MyDataset(Dataset):
    def __init__(self, his_datas, his_label, output_size, feature_size, seq_num, time_of_day):
        self.his_datas = his_datas  #【N，1080，X】
        # self.sta_datas = sta_datas  #【N，26，Y】
        self.his_label = his_label  #【N，1080，1】
        self.output_size = output_size  # 输出长度24
        self.feature_size = feature_size  # 卷积塔时序特征数量
        # self.static_feature_size = static_feature_size  # 特征塔天粒度/静态特征数量
        self.seq_num = seq_num  # 窗口大小
        self.time_of_day = time_of_day  # 每天24小时
         
        self.site_num = his_datas.shape[0]  # 站点数量
        self.time_num = his_datas.shape[1] // time_of_day  - (seq_num + 3) # 单个站点的样本数量：26-15=11个样本
        self.sample_num = self.time_num * self.site_num  # 总样本数量：32*1080=3w
        # print(his_datas.shape)
        print('单个样本数量：', self.time_num)
        print('站点数量：', self.site_num)
        print('总样本数量：', self.sample_num)
        print("a", his_datas.shape, his_label.shape)
        
    def __getitem__(self, index): # 0-3w
        # 是第几个样本？
        cls_indx, time_indx = divmod(index, self.time_num)
        start_index = time_indx * self.time_of_day
        end_index = (time_indx + self.seq_num) * self.time_of_day
        # [站点,小时粒度序列,小时粒度特征]
        tmp_data = self.his_datas[cls_indx, start_index:end_index, 0:self.feature_size].astype(float)  # [0, 14*24, time_feature_size]
        sample_time_data = torch.tensor(tmp_data, dtype=torch.float32)
        # [站点,天粒度序列,天粒度特征]
        # static_data = self.sta_datas[cls_indx, static_index:static_index+1, 0:self.static_feature_size].astype(float)  # [0, 1, time_feature_size]
        # sample_static_data = torch.tensor(static_data, dtype=torch.float32)
        # [站点,序列,1]
        label_start = end_index   #理想情况，不加self.time_of_day，即t日结束时预测t+1日的流量
        label_end = label_start + self.output_size
        target_label = self.his_label[cls_indx, label_start:label_end, 0:1].astype(float)
        sample_labels = torch.tensor(target_label, dtype=torch.float32)
        
        return sample_time_data, sample_labels
    
    def __len__(self):
        return self.sample_num

In [5]:
def train_test_split(all_data):  # 56天
    tmp_data_info = np.array(all_data)
    # sta_data_info = np.array(sta_data)
    # 当前总时长为138天，4.15-8.30
    train_start_idx = 0
    train_end_idx = 76 * 24 
    val_start_idx = 76 * 24
    val_end_idx = 107 * 24 
    test_start_idx = 107 * 24
    test_end_idx = 138 * 24 
    # train_start_sta_idx = 0
    # train_end_sta_idx = 18
    # val_end_sta_idx = 22
    # test_end_sta_idx = 26
    
#     train_start_idx = 0
#     train_end_idx = 38 * 24  # 9
#     val_start_idx = (38 - 30) * 24  # 13使用14，14使用15
#     val_end_idx = 42 * 24  # 4
#     test_start_idx = (42 - 30) * 24
#     test_end_idx = 49 * 24  # 7
    
    train_data = tmp_data_info[:, train_start_idx:train_end_idx, :]  # 所有特征
    # train_data_sta = sta_data_info[:, train_start_sta_idx:train_end_sta_idx, :]
    train_label = tmp_data_info[:, train_start_idx:train_end_idx, 0:1]
    val_data = tmp_data_info[:, val_start_idx:val_end_idx, :]
    # val_data_sta = sta_data_info[:, train_end_sta_idx:val_end_sta_idx, :]    
    val_label = tmp_data_info[:, val_start_idx:val_end_idx, 0:1]
    test_data = tmp_data_info[:, test_start_idx:test_end_idx, :]
    # test_data_sta = sta_data_info[:, val_end_sta_idx:test_end_sta_idx, :]  
    test_label = tmp_data_info[:, test_start_idx:test_end_idx, 0:1]
    return train_data, train_label, val_data, val_label, test_data, test_label
    # return train_data, train_data_sta, train_label, val_data, val_data_sta, val_label, test_data, test_data_sta, test_label



def load_data(all_data, batch_size):
    train_data, train_label, val_data, val_label, test_data, test_label = train_test_split(all_data)
    
    # 创建数据集
    train_dataset = MyDataset(his_datas=train_data, his_label=train_label, 
                             output_size=24, feature_size=8, seq_num=7, time_of_day=24)
    
    # 创建训练样本索引
    n_train = len(train_dataset)
    indices = list(range(n_train))
    np.random.shuffle(indices)
    split_point = int(n_train * 0.4)
    train_indices = indices[:split_point]
    
    # 创建采样器
    train_sampler = SubsetRandomSampler(train_indices)
    
    # 创建数据加载器
    train_dataloader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        sampler=train_sampler,
        pin_memory=True  # 加速GPU数据传输
    )
    
    # 验证和测试集保持完整
    val_dataset = MyDataset(his_datas=val_data, his_label=val_label, 
                           output_size=24, feature_size=8, seq_num=7, time_of_day=24)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    test_dataset = MyDataset(his_datas=test_data, his_label=test_label, 
                             output_size=24, feature_size=8, seq_num=7, time_of_day=24)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_dataloader, val_dataloader, test_dataloader



In [6]:
# Dlinear模型：简易baseline
def setup_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] =str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.daterministic = True

class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        # print(front.shape, x.shape, end.shape)
        x = torch.cat([front, x, end], dim=1)
        
        x = self.avg(x.permute(0, 2, 1))
        # print(x.shape)
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

class DLinear(nn.Module):
    """
    Decomposition-Linear
    """
    def __init__(self, seq_len, pred_len, individual, channels):
        super(DLinear, self).__init__()
        self.seq_len = seq_len
        self.pred_len = pred_len

        # Decompsition Kernel Size
        kernel_size = 49
        self.decompsition = series_decomp(kernel_size)
        self.individual = individual
        self.channels = channels

        if self.individual:
            self.Linear_Seasonal = nn.ModuleList()
            self.Linear_Trend = nn.ModuleList()
            
            for i in range(self.channels):
                self.Linear_Seasonal.append(nn.Linear(self.seq_len,self.pred_len))
                self.Linear_Trend.append(nn.Linear(self.seq_len,self.pred_len))

                # Use this two lines if you want to visualize the weights
                self.Linear_Seasonal[i].weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
                self.Linear_Trend[i].weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
        else:
            self.Linear_Seasonal = nn.Linear(self.seq_len,self.pred_len)
            self.Linear_Trend = nn.Linear(self.seq_len,self.pred_len)
            
            # Use this two lines if you want to visualize the weights
            self.Linear_Seasonal.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
            self.Linear_Trend.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))

    def forward(self, x):
        # x: [Batch, Input length, Channel]
        seasonal_init, trend_init = self.decompsition(x)
        seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)
        if self.individual:
            seasonal_output = torch.zeros([seasonal_init.size(0),seasonal_init.size(1),self.pred_len],dtype=seasonal_init.dtype).to(seasonal_init.device)
            trend_output = torch.zeros([trend_init.size(0),trend_init.size(1),self.pred_len],dtype=trend_init.dtype).to(trend_init.device)
            for i in range(self.channels):
                seasonal_output[:,i,:] = self.Linear_Seasonal[i](seasonal_init[:,i,:])
                trend_output[:,i,:] = self.Linear_Trend[i](trend_init[:,i,:])
        else:
            seasonal_output = self.Linear_Seasonal(seasonal_init)
            trend_output = self.Linear_Trend(trend_init)

        x = seasonal_output + trend_output
        x = x.permute(0,2,1) # to [Batch, Output length, Channel]

        return x[:,:,0]

class PeakHuberLoss(nn.Module):
    def __init__(self):
        super(PeakHuberLoss, self).__init__()
    def forward(self, y_pred, y_true, delta = 5):
        # y_pred: [B, 24, 1]; y_true: [B, 24, 1]
        # 标准化形状，确保可广播
        if y_pred.ndim == 2:
            y_pred = y_pred.unsqueeze(-1)
        if y_true.ndim == 2:
            y_true = y_true.unsqueeze(-1)
        error = y_true - y_pred
        peak_mask = (y_true >= 5)
        # 让空集合时保持为张量而不是 Python float
        if torch.any(peak_mask):
            peak_err = error[peak_mask]
            peak_loss = torch.where(torch.abs(peak_err) <= delta,
                                    0.5 * peak_err**2,
                                    delta * (torch.abs(peak_err) - 0.5 * delta)).mean()
        else:
            peak_loss = torch.zeros((), device=error.device)
        non_peak_mask = ~peak_mask
        if torch.any(non_peak_mask):
            non_peak_err = error[non_peak_mask]
            non_peak_loss = torch.abs(non_peak_err).mean()
        else:
            non_peak_loss = torch.zeros((), device=error.device)
        total_loss = peak_loss * 2 + non_peak_loss
        return total_loss  # 返回单个标量张量


In [7]:
setup_seed(12345)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
output_sizes = 24

# device = 'cpu'
print('device:', device)
print(all_data.shape)
# print(static_all_data.shape)

# 加载数据
train_dataloader, val_dataloader, test_dataloader = load_data(all_data[:, :, :], 1024)

device: cuda
(753, 3312, 8)
单个样本数量： 66
站点数量： 753
总样本数量： 49698
a (753, 1824, 8) (753, 1824, 1)
单个样本数量： 21
站点数量： 753
总样本数量： 15813
a (753, 744, 8) (753, 744, 1)
单个样本数量： 21
站点数量： 753
总样本数量： 15813
a (753, 744, 8) (753, 744, 1)


In [8]:
def train(train_dataloader, val_dataloader, model_save_path, epochs=50, lr=0.001):
    """训练模型"""
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # 初始化模型
    model = DLinear(seq_len=168, pred_len=24, individual=True, channels=8).to(device)
    model.train()
    
    # 损失函数和优化器
    criterion = PeakHuberLoss().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
    
    # 早停参数
    min_epochs = 10
    max_es_epoch = 10
    min_val_loss = float('inf')
    es_cnt = 0
    
    for epoch in tqdm(range(epochs)):
        model.train()
        train_losses = []
        
        for (seq, label) in train_dataloader:
            seq = seq.to(device)
            label = label.to(device)
            if label.shape[0] != 1024:
                continue
            optimizer.zero_grad()
            y_pred = model(seq)
            loss = criterion(y_pred, label)
            train_losses.append(loss.item())
            loss.backward()
            optimizer.step()
        
        train_loss_avg = sum(train_losses) / len(train_losses) if train_losses else 0

        # 每2个epoch进行验证
        if epoch % 2 == 0:
            model.eval()
            val_losses = []
            with torch.no_grad():
                for (seq, label) in val_dataloader:
                    seq = seq.to(device)
                    label = label.to(device)
                    if label.shape[0] != 1024:
                        continue
                    y_pred = model(seq)
                    loss = criterion(y_pred, label)
                    val_losses.append(loss.item())

            val_loss_avg = sum(val_losses) / len(val_losses) if val_losses else 0
            print(f'Epoch {epoch:03d} train_loss {train_loss_avg:.6f} val_loss {val_loss_avg:.6f}')

            if val_loss_avg < min_val_loss:
                min_val_loss = val_loss_avg
                es_cnt = 0
                torch.save(model.state_dict(), model_save_path)
                print(f'保存最佳模型，验证损失: {val_loss_avg:.6f}')
            else:
                es_cnt += 1
                if es_cnt >= max_es_epoch and epoch >= min_epochs:
                    print('触发早停机制！')
                    break
    
    return model


def test(test_dataloader, model_save_path):
    """测试模型，返回预测值和真实值"""
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # 加载模型
    model = DLinear(seq_len=168, pred_len=24, individual=True, channels=8).to(device)
    model.load_state_dict(torch.load(model_save_path))
    
    criterion = PeakHuberLoss().to(device)
    
    # 初始化存储
    test_losses = []
    true_values = []
    pred_values = []
    
    # 测试循环
    model.eval()
    with torch.no_grad():
        for test_time_data, test_labels in test_dataloader:
            test_time_data = test_time_data.to(device)
            test_labels = test_labels.to(device)
            if test_labels.shape[0] != 1024:
                continue
            # 前向传播
            test_forecasts = model(test_time_data)
            
            # 计算损失
            test_loss = criterion(test_forecasts, test_labels)
            test_losses.append(test_loss.item())
            
            # 存储真实值和预测值
            true_values.append(test_labels.cpu().numpy())
            pred_values.append(test_forecasts.cpu().numpy())
    
    # 计算平均损失
    test_loss_avg = sum(test_losses) / len(test_losses) if test_losses else 0
    print(f'Test Loss: {test_loss_avg:.6f}')
    
    return pred_values, true_values


def evaluate_metrics(pred_values, true_values):
    """评估测试集的 MSE / MAPE / WMAPE（仅统计真值>5的样本）"""
    import numpy as np
    
    if not pred_values or not true_values:
        print("pred_values / true_values 为空，请先运行测试循环。")
        return
    
    y_pred = np.concatenate(pred_values, axis=0)  # [N, 24, 1]
    y_true = np.concatenate(true_values, axis=0)  # [N, 24, 1]
    # print(y_pred.shape)
    # 去掉最后一个特征维度
    # y_pred = y_pred.squeeze(-1)  # [N, 24]
    # y_true = y_true.squeeze(-1)  # [N, 24]

    def compute_metrics_gt5(y_true_slice, y_pred_slice, gt_min=5):
        """仅在真值>gt_min的样本上计算指标"""
        mask = y_true_slice > gt_min
        if not np.any(mask):
            return float('nan'), float('nan'), float('nan')
        yt = y_true_slice[mask]
        yp = y_pred_slice[mask]
        mse = float(np.mean((yp - yt) ** 2))
        mape = float(np.mean(np.abs((yp - yt) / yt)))
        denom = float(np.sum(np.abs(yt)))
        wmape = float(np.sum(np.abs(yp - yt)) / denom) if denom > 0 else float('nan')
        return mse, mape, wmape

    # 定义时段索引
    morning_idx = np.array([7, 8, 9])
    evening_idx = np.array([18, 19, 20])
    all_idx = np.arange(24)

    # 早峰（仅真值>5）
    mse_morning, mape_morning, wmape_morning = compute_metrics_gt5(
        y_true[:, morning_idx].reshape(-1), y_pred[:, morning_idx].reshape(-1)
    )
    # 晚峰（仅真值>5）
    mse_evening, mape_evening, wmape_evening = compute_metrics_gt5(
        y_true[:, evening_idx].reshape(-1), y_pred[:, evening_idx].reshape(-1)
    )
    # 全天（仅真值>5）
    mse_all, mape_all, wmape_all = compute_metrics_gt5(
        y_true[:, all_idx].reshape(-1), y_pred[:, all_idx].reshape(-1)
    )

    print("\n=== Test Metrics (y_true > 5 only) ===")
    print(f"Morning 7-9   -> MSE: {mse_morning:.4f}, MAPE: {mape_morning:.4f}, WMAPE: {wmape_morning:.4f}")
    print(f"Evening 18-20 -> MSE: {mse_evening:.4f}, MAPE: {mape_evening:.4f}, WMAPE: {wmape_evening:.4f}")
    print(f"All-day 0-23  -> MSE: {mse_all:.4f}, MAPE: {mape_all:.4f}, WMAPE: {wmape_all:.4f}")
    
    return {
        'morning': {'mse': mse_morning, 'mape': mape_morning, 'wmape': wmape_morning},
        'evening': {'mse': mse_evening, 'mape': mape_evening, 'wmape': wmape_evening},
        'all_day': {'mse': mse_all, 'mape': mape_all, 'wmape': wmape_all}
    }


In [9]:
# 完整的训练、测试、评估流程

# 1. 训练模型
model_save_path = 'pred_model/net_divvy_DLinear_2.pth'
print("开始训练...")
trained_model = train(train_dataloader, val_dataloader, model_save_path, epochs=50, lr=0.001)

# 2. 测试模型
print("\n开始测试...")
pred_values, true_values = test(test_dataloader, model_save_path)
print(len(pred_values), len(true_values))
# 3. 评估指标
print("\n评估指标...")
metrics = evaluate_metrics(pred_values, true_values)


开始训练...


  2%|▏         | 1/50 [00:01<00:58,  1.20s/it]

Epoch 000 train_loss 54.712150 val_loss 24.123800
保存最佳模型，验证损失: 24.123800


  6%|▌         | 3/50 [00:02<00:42,  1.12it/s]

Epoch 002 train_loss 39.876983 val_loss 20.588317
保存最佳模型，验证损失: 20.588317


...

Epoch 046 train_loss 28.987665 val_loss 17.128441


 98%|█████████▊| 49/50 [00:43<00:00,  1.04it/s]

Epoch 048 train_loss 28.393745 val_loss 16.791861
保存最佳模型，验证损失: 16.791861


100%|██████████| 50/50 [00:44<00:00,  1.12it/s]



开始测试...
Test Loss: 17.816539
15 15

评估指标...

=== Test Metrics (y_true > 5 only) ===
Morning 7-9   -> MSE: 71.6610, MAPE: 0.5797, WMAPE: 0.5838
Evening 18-20 -> MSE: 45.1358, MAPE: 0.5338, WMAPE: 0.5166
All-day 0-23  -> MSE: 49.2837, MAPE: 0.4523, WMAPE: 0.4187
